# 大井競馬データ探索ノートブック

このノートブックでは、収集した大井競馬のデータを探索・分析します。

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# プロジェクトルートをパスに追加
sys.path.append(str(Path.cwd().parent))

from src.data_collection.database import OiKeibaDatabase

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style('whitegrid')

# データベース接続
db = OiKeibaDatabase()
print('データベースに接続しました')

## データの基本情報

In [ ]:
# データ読み込み
df = db.get_race_data()

print(f'総レコード数: {len(df):,}')
print(f'期間: {df["race_date"].min()} ～ {df["race_date"].max()}')
print(f'ユニークレース数: {df["race_id"].nunique():,}')
print(f'ユニーク馬数: {df["horse_name"].nunique():,}')

# データの概要
df.info()

In [ ]:
# 最初の数行を表示
df.head()

## 欠損値の確認

In [ ]:
# 欠損値の確認
missing_data = df.isnull().sum()
missing_percent = 100 * missing_data / len(df)

missing_table = pd.DataFrame({
    '欠損数': missing_data,
    '欠損率(%)': missing_percent
})

missing_table[missing_table['欠損数'] > 0].sort_values('欠損数', ascending=False)

## 基本統計

In [ ]:
# 数値データの基本統計
numeric_columns = ['course_length', 'finish_position', 'horse_weight', 'odds', 'popularity']
df[numeric_columns].describe()

## データ可視化

In [ ]:
# 着順分布
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
df['finish_position'].hist(bins=16, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('着順分布')
plt.xlabel('着順')
plt.ylabel('頻度')

plt.subplot(1, 2, 2)
df['popularity'].hist(bins=16, alpha=0.7, color='lightcoral', edgecolor='black')
plt.title('人気分布')
plt.xlabel('人気')
plt.ylabel('頻度')

plt.tight_layout()
plt.show()

In [ ]:
# オッズ分布（対数スケール）
plt.figure(figsize=(10, 6))

# 異常値を除外
odds_filtered = df[(df['odds'] > 0) & (df['odds'] <= 100)]['odds']

plt.hist(odds_filtered, bins=50, alpha=0.7, color='gold', edgecolor='black')
plt.title('オッズ分布')
plt.xlabel('オッズ')
plt.ylabel('頻度')
plt.yscale('log')
plt.show()

In [ ]:
# 人気と着順の関係
plt.figure(figsize=(12, 8))

# 散布図
plt.subplot(2, 2, 1)
plt.scatter(df['popularity'], df['finish_position'], alpha=0.5)
plt.title('人気 vs 着順')
plt.xlabel('人気')
plt.ylabel('着順')

# 人気別平均着順
plt.subplot(2, 2, 2)
popularity_avg = df.groupby('popularity')['finish_position'].mean()
popularity_avg.plot(kind='bar', color='lightblue')
plt.title('人気別平均着順')
plt.xlabel('人気')
plt.ylabel('平均着順')
plt.xticks(rotation=45)

# 人気別勝率
plt.subplot(2, 2, 3)
win_rate_by_popularity = df.groupby('popularity').apply(
    lambda x: (x['finish_position'] == 1).sum() / len(x) * 100
)
win_rate_by_popularity.plot(kind='bar', color='lightgreen')
plt.title('人気別勝率')
plt.xlabel('人気')
plt.ylabel('勝率 (%)')
plt.xticks(rotation=45)

# コース距離分布
plt.subplot(2, 2, 4)
df['course_length'].value_counts().sort_index().plot(kind='bar', color='orange')
plt.title('コース距離分布')
plt.xlabel('距離 (m)')
plt.ylabel('レース数')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 騎手・調教師分析

In [ ]:
# 騎手別成績（上位10名）
jockey_stats = df.groupby('jockey_name').agg({
    'finish_position': ['count', 'mean'],
    'horse_name': lambda x: (df.loc[x.index, 'finish_position'] == 1).sum()
}).round(2)

jockey_stats.columns = ['出走数', '平均着順', '勝利数']
jockey_stats['勝率'] = (jockey_stats['勝利数'] / jockey_stats['出走数'] * 100).round(1)

# 最低出走数でフィルタリング
min_races = 20
top_jockeys = jockey_stats[jockey_stats['出走数'] >= min_races].sort_values('勝率', ascending=False).head(10)

print('騎手別成績（上位10名）')
print(top_jockeys)

In [ ]:
# 調教師別成績（上位10名）
trainer_stats = df.groupby('trainer_name').agg({
    'finish_position': ['count', 'mean'],
    'horse_name': lambda x: (df.loc[x.index, 'finish_position'] == 1).sum()
}).round(2)

trainer_stats.columns = ['出走数', '平均着順', '勝利数']
trainer_stats['勝率'] = (trainer_stats['勝利数'] / trainer_stats['出走数'] * 100).round(1)

# 最低出走数でフィルタリング
top_trainers = trainer_stats[trainer_stats['出走数'] >= min_races].sort_values('勝率', ascending=False).head(10)

print('調教師別成績（上位10名）')
print(top_trainers)

## 馬体重と成績の関係

In [ ]:
# 馬体重を区間に分けて分析
df_weight = df[df['horse_weight'] > 0].copy()
df_weight['weight_category'] = pd.cut(df_weight['horse_weight'], 
                                    bins=[0, 450, 480, 510, 1000], 
                                    labels=['軽量', '標準', '重量', '超重量'])

weight_analysis = df_weight.groupby('weight_category').agg({
    'finish_position': 'mean',
    'horse_name': lambda x: (df_weight.loc[x.index, 'finish_position'] == 1).sum() / len(x) * 100
}).round(2)

weight_analysis.columns = ['平均着順', '勝率(%)']
print('馬体重別成績')
print(weight_analysis)

## 天候・馬場状態の影響

In [ ]:
# 天候別分析
if 'weather' in df.columns and df['weather'].notna().any():
    weather_analysis = df.groupby('weather').agg({
        'finish_position': ['count', 'mean']
    }).round(2)
    
    weather_analysis.columns = ['レース数', '平均着順']
    print('天候別分析')
    print(weather_analysis)
else:
    print('天候データがありません')

# 馬場状態別分析
if 'track_condition' in df.columns and df['track_condition'].notna().any():
    track_analysis = df.groupby('track_condition').agg({
        'finish_position': ['count', 'mean']
    }).round(2)
    
    track_analysis.columns = ['レース数', '平均着順']
    print('
馬場状態別分析')
    print(track_analysis)
else:
    print('馬場状態データがありません')

## まとめ

このノートブックでの分析結果をもとに、以下の特徴量が予想に有効であることが分かりました：

1. **人気**: 人気と着順には強い相関がある
2. **騎手・調教師**: 成績に差がある
3. **馬体重**: 適正体重がある可能性
4. **コース距離**: 馬の得意距離がある
5. **オッズ**: 市場の評価を反映

次のステップ：
- より詳細な特徴量エンジニアリング
- 馬の過去成績を考慮した特徴量
- 時系列要素を考慮した分析